Author: Omar El Malki (omar.elmalki@epfl.ch)

### Data Exploration
- ROCStories
- ConceptNet 5.7.0
- ATOMIC-2020

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [3]:
roc_stories_csv = '../../generated/conceptnet-node-extraction/ROCStories_with_cn_heads.csv'
roc_stories_df = pd.read_csv(roc_stories_csv, sep='\t', header=0)
roc_stories_df = roc_stories_df.iloc[:10]

In [4]:
roc_stories_df

,Unnamed: 0,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,storyid,storytitle,sentence1,sentence2,sentence3,...,triples1,triples2,triples3,triples4,triples5,cn_nodes1,cn_nodes2,cn_nodes3,cn_nodes4,cn_nodes5
0,0,0,0,0,0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,...,"[[""Dan 's parents"", 'were', 'overweight']]","[['Dan', 'was', 'overweight'], ['Dan', 'was', 'as well']]","[['The doctors', 'told', ""Dan 's parents""], ['The doctors', 'told', 'it was unhealthy'], ['it', 'was', 'unhealthy']]","[[""Dan 's parents"", 'decided', 'to make a change'], [""Dan 's parents"", 'make', 'a change']]","[[""Dan 's parents"", 'got', ""Dan 's parents and Dan""], [""Dan 's parents"", 'got', 'on a diet']]","{'verb': set(), 'entity': {'dan', 'parents', 'overweight'}, 'verb_lemma': set(), 'entity_lemma': {'dan', 'overweight', 'parent'}}","{'verb': set(), 'entity': {'dan', 'well', 'as well', 'overweight'}, 'verb_lemma': set(), 'entity_lemma': {'dan', 'well', 'as well', 'overweight'}}","{'verb': {'told'}, 'entity': {'doctors', 'dan', 'parents', 'unhealthy'}, 'verb_lemma': {'tell'}, 'entity_lemma': {'dan', 'doctor', 'unhealthy', 'parent'}}","{'verb': {'make', 'decided'}, 'entity': {'dan', 'parents', 'make', 'change'}, 'verb_lemma': {'make', 'decide'}, 'entity_lemma': {'dan', 'make', 'parent', 'change'}}","{'verb': {'got'}, 'entity': {'dan', 'parents', 'diet'}, 'verb_lemma': {'get'}, 'entity_lemma': {'dan', 'diet', 'parent'}}"
1,1,1,1,1,1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,...,"[['Carrie', 'learned', 'just'], ['Carrie', 'learned', 'how to ride a bike'], ['Carrie', 'ride', 'how'], ['Carrie', 'ride', 'a bike']]","[['Carrie', 'have', ""n't""], ['Carrie', 'have', 'a bike of Carrie own']]","[['Carrie', 'sneak', 'would'], ['Carrie', 'sneak', ""rides on Carrie sister 's bike""]]","[['Carrie', 'got', 'nervous'], ['Carrie', 'got', 'on a hill'], ['Carrie', 'crashed', 'into a wall']]","[['Carrie', 'got', 'a deep gash on Carrie leg']]","{'verb': {'learned', 'ride'}, 'entity': {'bike', 'ride', 'carrie', 'how to'}, 'verb_lemma': {'ride', 'learn'}, 'entity_lemma': {'bike', 'ride', 'carrie', 'how to'}}","{'verb': set(), 'entity': {'bike', 'carrie', ""n't""}, 'verb_lemma': set(), 'entity_lemma': {'bike', 'carrie'}}","{'verb': {'sneak'}, 'entity': {'bike', 'carrie', 'rides', 'sister', 'would'}, 'verb_lemma': {'sneak'}, 'entity_lemma': {'bike', 'sister', 'carrie', 'ride on', 'ride', 'would'}}","{'verb': {'crashed', 'got'}, 'entity': {'nervous', 'carrie', 'wall', 'hill'}, 'verb_lemma': {'get', 'crash'}, 'entity_lemma': {'nervous', 'carrie', 'wall', 'hill'}}","{'verb': {'got'}, 'entity': {'leg', 'deep', 'carrie', 'gash'}, 'verb_lemma': {'get'}, 'entity_lemma': {'leg', 'deep', 'carrie', 'gash'}}"
2,2,2,2,2,2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long walk.,"After walking for over a mile, something happened.",...,"[['Morgan', 'enjoyed', 'long walks on the beach']]","[['Morgan and Morgan boyfriend', 'decided', 'to go for a long walk'], ['Morgan and Morgan boyfriend', 'go', 'for a long walk']]","[['something', 'happened', 'After walking for over a mile']]","[['Morgan', 'decided', 'to propose to her boyfriend'], ['Morgan', 'propose', 'to her boyfriend']]","[['her boyfriend', 'was', ""upset Morgan did n't propose to her boyfriend first""], [""Morgan did n't propose to her boyfriend first"", 'upset', 'her boyfriend'], ['Morgan', 'propose', ""n't""], ['Morgan', 'propose', 'to her boyfriend'], ['Morgan', 'propose', 'first']]","{'verb': {'enjoyed'}, 'entity': {'beach', 'long', 'morgan', 'walks'}, 'verb_lemma': {'enjoy'}, 'entity_lemma': {'long', 'beach', 'walk', 'morgan', 'walk on'}}","{'verb': {'decided', 'go'}, 'entity': {'long', 

In [22]:
del roc_stories_triples_df['srl_r1']

In [43]:
def triple_to_entity_set(triples):
    triples = ast.literal_eval(triples)
    s = []
    for triple in triples:
        s.append(triple[0].strip())
        s.append(triple[2].strip())
    return set(s)

In [44]:
def lemmatize_set(input_set):
    res = []
    for e in input_set:
        doc = nlp(e)
        res.append(" ".join([token.lemma_ for token in doc]))
    return set(res)

In [45]:
def extract_cn_nodes(input_set):
    res = []
    for e in input_set:
        words = e.lower().split()
        two_grams = [" ".join(words[i:i+2]) for i in range(len(words)-1)]
        words += two_grams
        res += [e.strip() for e in words if e.strip() in entities]
    return set(res)

In [46]:
def triple_to_verb_set(triples):
    triples = ast.literal_eval(triples)
    s = [triple[1].strip() for triple in triples]
    return set(s)

In [51]:
def extract_cn_nodes_from_triples(triples):
    verb_set = triple_to_verb_set(triples)
    entity_set = triple_to_entity_set(triples)
    lemmatized_verb_set = lemmatize_set(verb_set)
    lemmatized_entity_set = lemmatize_set(entity_set)
    
    verb_cn_nodes = extract_cn_nodes(verb_set)
    entity_cn_nodes = extract_cn_nodes(entity_set)
    lemmatized_verb_cn_nodes = extract_cn_nodes(lemmatized_verb_set)
    lemmatized_entity_cn_nodes = extract_cn_nodes(lemmatized_entity_set)
    
    cn_nodes = {}
    cn_nodes['verb'] = verb_cn_nodes
    cn_nodes['entity'] = entity_cn_nodes
    cn_nodes['verb_lemma'] = lemmatized_verb_cn_nodes
    cn_nodes ['entity_lemma'] = lemmatized_entity_cn_nodes
    
    return cn_nodes

In [28]:
triples = """[['The doctors', 'told', "Dan 's parents"], ['The doctors', 'told', 'it was unhealthy'], ['it', 'was', 'unhealthy']]"""

In [52]:
extract_cn_nodes_from_triples(triples)

{'verb': {'told'},
 'entity': {'dan', 'doctors', 'parents', 'unhealthy'},
 'verb_lemma': {'tell'},
 'entity_lemma': {'dan', 'doctor', 'parent', 'unhealthy'}}

In [47]:
verb_set = triple_to_verb_set(triples)
print(verb_set)
entity_set = triple_to_entity_set(triples)
print(entity_set)
lemmatized_verb_set = lemmatize_set(verb_set)
print(lemmatized_verb_set)
lemmatized_entity_set = lemmatize_set(entity_set)
print(lemmatized_entity_set)

{'was', 'told'}
{'The doctors', 'unhealthy', "Dan 's parents", 'it', 'it was unhealthy'}
{'tell', 'be'}
{'unhealthy', "Dan 's parent", '-PRON- be unhealthy', '-PRON-', 'the doctor'}


In [48]:
verb_cn_nodes = extract_cn_nodes(verb_set)
print(verb_cn_nodes)
entity_cn_nodes = extract_cn_nodes(entity_set)
print(entity_cn_nodes)
lemmatized_verb_cn_nodes = extract_cn_nodes(lemmatized_verb_set)
print(lemmatized_verb_cn_nodes)
lemmatized_entity_cn_nodes = extract_cn_nodes(lemmatized_entity_set)
print(lemmatized_entity_cn_nodes)

{'told'}
{'unhealthy', 'doctors', 'parents', 'dan'}
{'tell'}
{'unhealthy', 'parent', 'doctor', 'dan'}


In [49]:
cn_nodes = {}
cn_nodes['verb'] = verb_cn_nodes
cn_nodes['entity'] = entity_cn_nodes
cn_nodes['verb_lemma'] = lemmatized_verb_cn_nodes
cn_nodes ['entity_lemma'] = lemmatized_entity_cn_nodes

In [50]:
cn_nodes

{'verb': {'told'},
 'entity': {'dan', 'doctors', 'parents', 'unhealthy'},
 'verb_lemma': {'tell'},
 'entity_lemma': {'dan', 'doctor', 'parent', 'unhealthy'}}

In [ ]:
# def triple_to_str(triples):
#     triples = ast.literal_eval(triples)
#     s = ""
#     for triple in triples:
#         s += " ".join(triple) + " "
#     return s.strip()

In [ ]:
# roc_stories_triples_df

In [ ]:
# extract_entities(triple_to_str("""[['The doctors', 'told', "Dan 's parents"], ['The doctors', 'told', 'it was unhealthy'], ['it', 'was', 'unhealthy']]"""))

In [56]:
for n in range(1, 6):
    roc_stories_triples_df[f'cn_nodes{n}'] = roc_stories_triples_df[f'triples{n}'].progress_apply(extract_cn_nodes_from_triples)

100%|██████████| 10/10 [00:00<00:00, 37.71it/s]


In [57]:
roc_stories_triples_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,...,triples1,triples2,triples3,triples4,triples5,cn_nodes1,cn_nodes2,cn_nodes3,cn_nodes4,cn_nodes5
0,0,0,0,0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a change.,...,"[[""Dan 's parents"", 'were', 'overweight']]","[['Dan', 'was', 'overweight'], ['Dan', 'was', 'as well']]","[['The doctors', 'told', ""Dan 's parents""], ['The doctors', 'told', 'it was unhealthy'], ['it', 'was', 'unhealthy']]","[[""Dan 's parents"", 'decided', 'to make a change'], [""Dan 's parents"", 'make', 'a change']]","[[""Dan 's parents"", 'got', ""Dan 's parents and Dan""], [""Dan 's parents"", 'got', 'on a diet']]","{'verb': {}, 'entity': {'parents', 'overweight', 'dan'}, 'verb_lemma': {}, 'entity_lemma': {'parent', 'overweight', 'dan'}}","{'verb': {}, 'entity': {'well', 'as well', 'overweight', 'dan'}, 'verb_lemma': {}, 'entity_lemma': {'well', 'as well', 'overweight', 'dan'}}","{'verb': {'told'}, 'entity': {'unhealthy', 'doctors', 'parents', 'dan'}, 'verb_lemma': {'tell'}, 'entity_lemma': {'unhealthy', 'parent', 'doctor', 'dan'}}","{'verb': {'make', 'decided'}, 'entity': {'make', 'change', 'parents', 'dan'}, 'verb_lemma': {'make', 'decide'}, 'entity_lemma': {'make', 'parent', 'change', 'dan'}}","{'verb': {'got'}, 'entity': {'diet', 'parents', 'dan'}, 'verb_lemma': {'get'}, 'entity_lemma': {'parent', 'diet', 'dan'}}"
1,1,1,1,1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a wall.,...,"[['Carrie', 'learned', 'just'], ['Carrie', 'learned', 'how to ride a bike'], ['Carrie', 'ride', 'how'], ['Carrie', 'ride', 'a bike']]","[['Carrie', 'have', ""n't""], ['Carrie', 'have', 'a bike of Carrie own']]","[['Carrie', 'sneak', 'would'], ['Carrie', 'sneak', ""rides on Carrie sister 's bike""]]","[['Carrie', 'got', 'nervous'], ['Carrie', 'got', 'on a hill'], ['Carrie', 'crashed', 'into a wall']]","[['Carrie', 'got', 'a deep gash on Carrie leg']]","{'verb': {'learned', 'ride'}, 'entity': {'how to', 'carrie', 'bike', 'ride'}, 'verb_lemma': {'learn', 'ride'}, 'entity_lemma': {'how to', 'carrie', 'bike', 'ride'}}","{'verb': {}, 'entity': {'n't', 'carrie', 'bike'}, 'verb_lemma': {}, 'entity_lemma': {'n't', 'carrie', 'bike'}}","{'verb': {'sneak'}, 'entity': {'sister', 'rides', 'carrie', 'bike', 'would'}, 'verb_lemma': {'sneak'}, 'entity_lemma': {'sister', 'ride on', 'carrie', 'bike', 'ride', 'would'}}","{'verb': {'got', 'crashed'}, 'entity': {'nervous', 'carrie', 'hill', 'wall'}, 'verb_lemma': {'get', 'crash'}, 'entity_lemma': {'nervous', 'carrie', 'hill', 'wall'}}","{'verb': {'got'}, 'entity': {'gash', 'carrie', 'deep', 'leg'}, 'verb_lemma': {'get'}, 'entity_lemma': {'gash', 'carrie', 'deep', 'leg'}}"
2,2,2,2,2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long walk.,"After walking for over a mile, something happened.",Morgan decided to propose to her boyfriend.,...,"[['Morgan', 'enjoyed', 'long walks on the beach']]","[['Morgan and Morgan boyfriend', 'decided', 'to go for a long walk'], ['Morgan and Morgan boyfriend', 'go', 'for a long walk']]","[['something', 'happened', 'After walking for over a mile']]","[['Morgan', 'decided', 'to propose to her boyfriend'], ['Morgan', 'propose', 'to her boyfriend']]","[['her boyfriend', 'was', ""upset Morgan did n't propose to her boyfriend first""], [""Morgan did n't propose to her boyfriend first"", 'upset', 'her boyfriend'], ['Morgan', 'propose', ""n't""], ['Morgan', 'propose', 'to her boyfriend'], ['Morgan', 'propose', 'first']]","{'verb': {'enjoyed'}, 'entity': {'long', 'beach', 'morgan', 'walks'}, 'verb_lemma'

In [ ]:
# # Display nodes related to 'weight' in ConceptNet
# conceptnet_formatted_english_df[conceptnet_formatted_english_df['head'].str.contains('weight')]

In [ ]:
# # Display links between the nodes related to 'fast_food' and 'weight' in ConceptNet
# conceptnet_formatted_english_df[conceptnet_formatted_english_df['head'].str.contains('fast_food') & conceptnet_formatted_english_df['tail'].str.contains('weight')]

In [ ]:
# # Display nodes related to 'fast food' in ATOMIC
# atomic_train_df[atomic_train_df['head'].str.contains('fast food')]

In [ ]:
# # Display nodes related to 'weight' in ATOMIC
# atomic_train_df[atomic_train_df['head'].str.contains('weight')]

In [ ]:
# # Display links between the nodes related to 'fast food' and 'weight' in ATOMIC
# atomic_train_df[atomic_train_df['head'].str.contains('fast food') & atomic_train_df['tail'].str.contains('weight')]

In [ ]:
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.stem import PorterStemmer

In [ ]:
# lemmatizer = WordNetLemmatizer()

In [ ]:
# def lemmatize_sentence(sentence):
#     # Tokenize: Split the sentence into words
#     word_list = load_model(sentence)

#     # Lemmatize list of words and join
#     lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
#     return lemmatized_output

In [ ]:
# ps = PorterStemmer()

In [ ]:
# def stem_sentence(sentence):
#     # Tokenize: Split the sentence into words
#     word_list = nltk.word_tokenize(sentence)

#     # Lemmatize list of words and join
#     stemmed_output = ' '.join(set([ps.stem(w) for w in word_list]))
#     return stemmed_output

In [ ]:
verb_set = triple_to_verb_set(triples)

In [ ]:
stem_sentence(triple_to_str("""[['The doctors', 'told', "Dan 's parents"], ['The doctors', 'told', 'it was unhealthy'], ['it', 'was', 'unhealthy']]"""))